In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

import web_crawler.scraper_amazon as scraper_amazon

# Our generated code
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2

In [2]:
step_size = 10

#with open('/libs/data.json') as f:
#    json_data = json.load(f)

products = pd.read_pickle("./data/products_pd.pickle")

products['price_dollar'] = np.nan

i=0

In [3]:
products.iloc[i:i+step_size]

,created_t,product_name,quantity,packaging,brands,brands_tags,categories,categories_tags,origins,origins_tags,...,labels,purchase_places,stores,countries_en,main_category,energy_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,price_dollar
code,,,,,,,,,,,,,,,,,,,,,
0819377000120,1537556615,châtaignes pelées et rôties cesares,150 g,conserve,cesares,cesares,"Aliments et boissons à base de végétaux,Alimen...","en:plant-based-foods-and-beverages,en:plant-ba...",chine,chine,...,NaN,"canada,montreal",sami fruits,[Canada],en:plant-based-foods-and-beverages,1000.0,300.00,0,0,NaN
2000000072530,1533075976,Nestle Aguitas Manzana,300ml,botella de plastico,nestle,nestle,Agua embotellada,es:agua-embotellada,Cd. de Mexico,cd-de-mexico,...,ECOCE,walmart,"walmart,bodega aurrera,sams club",[Mexico],es:agua-embotellada,41.3,0.00,0,0,NaN
2000000074609,1536152379,Terrine de Chevreuil,180 gr,"Conserve,bocal,Verre,180gr","AJM,SARL AJM","ajm,sarl-ajm","Meat-based products,Terrine,Terrines de chevre...","en:meat-based-products,en:terrine,fr:terrines-...","France,Saveur Occitanie","france,saveur-occitanie",...,"Fait Maison,Carbon-footprint","Bagard,Gard,Occitanie,France","SARL AJM,AJM",[France],en:meat-based-products,1653.0,0.05,21,21,NaN
2000000074660,1536214006,TERRINE AUX CHÂTAIGNES,180 gr,"Conserve,Bocal,Verre,Verrine,180gr","AJM,Sarl AJM","ajm,sarl-ajm","Aliments et boissons à base de végétaux,Alimen...","en:plant-based-foods-and-beverages,en:plant-ba...","France,Saveur Occitanie","france,saveur-occitanie",...,Fait Maison,"Bagard,Gard,Occitanie,FRANCE,30140","AJM,Sarl AJM",[France],en:plant-based-foods-and-beverages,1653.0,0.05,21,21,NaN
2126744000002,1461504012,Bünder Bergkäse,NaN,Plastique,"Léger,Migros","leger,migros",Fromage,"en:dairies,en:fermented-foods,en:fermented-mil...",Suisse,suisse,...,NaN,NaN,Migros,[Switzerland],en:dairies,1010.0,142.70,12,17,NaN
2129746000001,1462721730,Minitaler,NaN,Plastique,"Léger,Migros","leger,migros",Fromage,"en:dairies,en:fermented-foods,en:fermented-mil...",Suisse,suisse,...,NaN,NaN,Migros,[Switzerland],en:dairies,1340.0,155.20,12,17,NaN
2141870000009,1461503777,Tilsiter doux,NaN,Plastique,"Léger,Migros","leger,migros",Fromage,"en:dairies,en:fermented-foods,en:fermented-mil...",NaN,NaN,...,NaN,NaN,Migros,[Switzerland],en:dairies,1000.0,145.80,8,13,NaN
2181121000006,1462721325,Plateau de fromages,NaN,Barquette plastique,"Léger,Migros","leger,migros",Fromage,"en:dairies,en:fermented-foods,en:fermented-mil...",NaN,NaN,...,NaN,NaN,Migros,[Switzerland],en:dairies,1120.0,152.10,14,19,NaN
22016065,1425215080,Jogourt Nature ferme,180 g,"Plastique,Alu","M-Classic,Migros, MClassic","m-classic,migros,mclassic",Yaourt,en:yaourt,NaN,NaN,...,NaN,NaN,Migros,[Switzerland],en:yaourt,270.0,57.80,1,1,NaN


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas

base_url = "https://produkte.migros.ch/sortiment?q=Tilsiter"

# To get the html contents
r = requests.get(base_url)
c = r.text

# To parse the html
soup = BeautifulSoup(c,"html.parser")


print(soup.select(".mui-js-product-list > li:nth-child(1) > a:nth-child(1) > div:nth-child(2) > div:nth-child(1)"))
# To extract the first and last page numbers
for product in soup.select(".mui-js-product-list"):
    print(product)


[]


In [ ]:
start_page = paging[1].text
last_page = paging[len(paging)-2].text
web_content_list = []

for page_number in range(int(start_page),int(last_page) + 1):
    
    # To form the url based on page numbers
    url = base_url+str(page_number)+"/.html"
    r = requests.get(base_url+str(page_number)+"/")
    c = r.content
    soup = BeautifulSoup(c,"html.parser")
    
    # To extract the Title and the Location
    placard_header = soup.find_all("header",{"class":"placardHeader"})
    
    # To extract the Rent, No of Beds and Phone Number
    placard_content = soup.find_all("section",{"class" :"placardContent"})
    
    # To process property by property by looping
    for item_header,item_content in zip(placard_header,placard_content):
      
      # To store the information to a dictionary
      web_content_dict = {}
      web_content_dict["Title"]=item_header.find("a",{"class":"placardTitle js-placardTitle "}).text.replace("\r","").replace("\n","")
      web_content_dict["Address"] = item_header.find("div",{"class":"location"}).text
      web_content_dict["Price"] = item_content.find("span",{"class":"altRentDisplay"}).text
      web_content_dict["Beds"] = item_content.find("span",{"class":"unitLabel"}).text
      web_content_dict["Phone"] = item_content.find("div",{"class":"phone"}).find("span").text
      
      # To store the dictionary to into a list
      web_content_list.append(web_content_dict)
    
# To make a dataframe with the list
df = pandas.DataFrame(web_content_list)

# To write the dataframe to a csv file
df.to_csv("Output.csv")

In [102]:
import requests  
from lxml import html

response = requests.get("https://produkte.migros.ch/tilsiter")

# Parse the body into a tree
soup = BeautifulSoup(response.content, "html.parser")

# Perform xpaths on the tree
print(soup.findAll("mui-product-tile-price-container mui-product-tile-price-container-shadow",{"id":"price-210406024000"})) # Get page title  


[]


In [87]:
print(elements[0].tag)

section
